## В данном блокноте выполняется предобработка датасета

Сокращение длинны контекста

In [ ]:
from datasets import load_dataset, load_from_disk #Загрузка датасета 

# ----------- for ds preprocess -----------
import nltk
nltk.data.path.clear()
nltk.data.path.append('C:/Users/nikita/Desktop/mlRes/nltk_data')
nltk.download('punkt', download_dir="C:/Users/nikita/Desktop/mlRes/nltk_data")
nltk.download('punkt_tab', download_dir="C:/Users/nikita/Desktop/mlRes/nltk_data")

print(nltk.data.path)

In [ ]:
# ----- | Параметры | -----

OUT_DATA_CACHE = "./prep_datasets" #Директория, с предобработанными датасетами
CACHE_DIR = "../myDoRA_repeat/cache_dir" #Директория с кешем модели
EVAL_MODEL_DIR_OUT = "./eval_models_out" #Для вывода логов

### Dataset - SQuAD 2.0

In [ ]:
#Загружаем датасет
datasetSQUAD2 = load_dataset("rajpurkar/squad_v2", cache_dir="./cdatasets")

val_dataset = datasetSQUAD2['validation'] 
train_dataset = datasetSQUAD2['train']

#for test pipeline
small_train_ds = train_dataset.select(range(32))
small_val_ds = val_dataset.select(range(32))

print(val_dataset.info)
print(train_dataset.info)

### Предобработка датасета 

Тестировал прямой проход модели на батче из 1 последовательности (1024 токена), работала, но видно с обратным распространением и шагом оптимизатора, памяти не хватает, следовательно,нужно сократить длинну последовательности до 256-512 токенов.

Возьму примерно 256 токенов (контекст обрезается по предложениям, берется текущее предыдушее и следущее).

Инференс на 1024 токена работает хорошо (причем я указываю что использую внимани "eager" для FT и "sdpa" для инференса, но не уверен, что оно корректно используется, не до конца протестировал)

In [ ]:
def preprocess_dataset_split_context(batch, neighbors = 1):
    """ Функция для обработки датасета SQuAD 2.0
    Для извлечения context, question, answer. И обрезки context до длинны 4-5 предложений.
    
    Статический срез, берет предыдушее предложение, текущее и следущее (index - neighbors : index + neighbors + 1)

    Args:
        batch : dict - словарь списков с соответствующими ключами dataset'a
    """
    #Есть строки, в которых нет ответов (в контексте не предоставлены ответы)
    #Необходимо, чтобы все ровно соответстовали тому, что было
    contexts = [] #Список контекстов
    questions = [] #Список воспросов
    answers = [] #Список ответов

    for cntxt, quest, answer in zip(batch['context'], batch['question'], batch['answers']):
        cntx_split_sentenses = nltk.sent_tokenize(cntxt, language='english') #Разбиваем контест по предложениям

        if len(answer['text']) > 0: #Ответ есть
            found = False
            for sent_index in range(len(cntx_split_sentenses)):
                if (answer['text'][0] in cntx_split_sentenses[sent_index]): #Если ответ содержится в предложении
                    start = max(0, sent_index - neighbors)
                    end = min(len(cntx_split_sentenses), sent_index + neighbors + 1)
                    res_text_context = " ".join(cntx_split_sentenses[start : end])

                    contexts.append(res_text_context)
                    questions.append(quest)
                    answers.append(answer['text'][0]) #Добавляем именно текст
                    found = True
                    break #ОБЯЗАТЕЛЬНО НУЖНО ВЫЙТИ из цикла
            if not found:
                contexts.append("") #Если не нашли, будет пустой контекст (не будем использовать для обучения и оценки)
                questions.append(quest)
                answers.append(answer['text'][0]) #Добавляем именно текст
        else:
            #Нужно добавить контекст как строку, а не как список ;D
            contexts.append(" ".join(cntx_split_sentenses[:3])) #Берем 3 первых предложения
            questions.append(quest)
            answers.append("") #Добавляем пустую строку
        
    assert len(contexts) == len(batch['context']), f"context not quals size | {len(contexts)} {len(batch['context'])}"
    assert len(questions) == len(batch['question']), f"question not quals size {len(questions)} {len(batch['question'])}"
    assert len(answers) == len(batch['answers']), f"answers not quals size {len(answers)} {len(batch['answers'])}"

    return { "context": contexts, "question": questions, "answers" : answers }

In [ ]:
train_short_ds = train_dataset.map(
    preprocess_dataset_split_context, 
    batched=True, 
    num_proc=1,
    remove_columns=train_dataset.column_names,
    cache_file_name="./cdatasets/train_ds_split_contex256.cache"
)

In [ ]:
val_short_ds = val_dataset.map(
    preprocess_dataset_split_context,
    batched=True,       
    num_proc=1,  
    remove_columns=val_dataset.column_names,  # Удаляем исходные колонки
    cache_file_name="./cdatasets/val_ds_split_contex256.cache"
)

### Удалим строки с пустным контекстом

In [ ]:
assert len(train_dataset) == len(train_short_ds), "size train_dataset and train_short_ds not equals"
assert len(val_dataset) == len(val_short_ds), "size val_dataset and val_short_ds not equals"
print(len(train_short_ds))
print(len(val_short_ds))

In [ ]:
# Подсчитаем сколько пустых контекстов в Train and Val
cnt_train = 0
for short in train_short_ds:
    if short['context'] == "":
        cnt_train += 1

cnt_val = 0
for short in val_short_ds:
    if short['context'] == "":
        cnt_val += 1

print(cnt_train) #214
print(cnt_val) #14

In [ ]:
train_ds_short_filter = train_short_ds.filter(lambda row: False if row['context'] == "" else True)
val_ds_short_filter = val_short_ds.filter(lambda row: False if row['context'] == "" else True)

In [ ]:
print(len(train_ds_short_filter))
print(len(val_ds_short_filter))

print(len(train_short_ds) - len(train_ds_short_filter))
print(len(val_short_ds) - len(val_ds_short_filter))

assert len(train_short_ds) - cnt_train == len(train_ds_short_filter), "Not equal Train"
assert len(val_short_ds) - cnt_val == len(val_ds_short_filter), "Not equal Train"

In [ ]:
train_ds_short_filter[0]

In [ ]:
val_ds_short_filter[0]

### Сохраним предобработанные датасеты

In [ ]:
train_ds_short_filter.save_to_disk(f"{OUT_DATA_CACHE}/train_ds_short")
val_ds_short_filter.save_to_disk(f"{OUT_DATA_CACHE}/val_ds_short")